In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## 什么时候用序贯模型
- 用于简单的层堆栈，而且每层正好一个输入和一个输出
- 当模型有多个输入或者多个输出事，不再适用
- 任何一层有多个输入或者多个输出， 不再适用
- 模型的某层需要共享， 不再适用
- 模型结构是非线性拓扑结构（如残差网络），不再适用

In [2]:
# 定义一个3层得序贯模型
model = tf.keras.Sequential([
    layers.Dense(2, activation='relu', name='layer1'),
    layers.Dense(3, activation='relu', name='layer2'),
    layers.Dense(4, name='layer3')
])

x = tf.ones((3,3))
y = model(x)
y.shape

TensorShape([3, 4])

In [3]:
y

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[ 0.8066562,  1.1773527, -1.041587 ,  0.6440198],
       [ 0.8066562,  1.1773527, -1.041587 ,  0.6440198],
       [ 0.8066562,  1.1773527, -1.041587 ,  0.6440198]], dtype=float32)>

In [4]:
model.layers

## 创建Sequential model

In [5]:
# 上述方式还可以这么写 通过add()方法
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu"))
model.add(layers.Dense(3, activation="relu"))
model.add(layers.Dense(4))

In [7]:
model.layers

In [8]:
# 当然还有pop方法
model.pop()
model.layers

In [9]:
# Sequential还可接受name参数
model = keras.Sequential(name="my_sequential")
model.add(layers.Dense(2, activation="relu", name="layer1"))
model.add(layers.Dense(3, activation="relu", name="layer2"))
model.add(layers.Dense(4, name="layer3"))

In [10]:
model.name

'my_sequential'

In [11]:
# 上述方式等价于函数式API写法
layer1 = layers.Dense(2, activation='relu', name='layer1')
layer2 = layers.Dense(3, activation='relu', name='layer2')
layer3 = layers.Dense(4, name='layer3')

x = tf.ones((3, 3))
y = layer3(layer2(layer1(x)))
y

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[ 0.02147376,  0.11640037, -0.24145482, -0.2028211 ],
       [ 0.02147376,  0.11640037, -0.24145482, -0.2028211 ],
       [ 0.02147376,  0.11640037, -0.24145482, -0.2028211 ]],
      dtype=float32)>

## 首先确认输入形状
- 权重的形状取决于输入的形状
- 创建某层之时，该层是没有权重的

In [14]:
l = layers.Dense(3)
l.weights # 权重为空

[]

In [15]:
x = tf.ones((1, 4))
y = l(x)
l.weights  # y = x * weights

[<tf.Variable 'dense_3/kernel:0' shape=(4, 3) dtype=float32, numpy=
 array([[-0.60233545,  0.22182667,  0.42413568],
        [ 0.60132   , -0.67690957, -0.5720699 ],
        [ 0.68353987,  0.17618954, -0.16014779],
        [-0.75985515, -0.67932016,  0.8499892 ]], dtype=float32)>,
 <tf.Variable 'dense_3/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

In [16]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)  # No weights at this stage!

In [17]:
try:
    model.weights
except Exception as e:
    print(e)

Weights for model sequential_2 have not yet been created. Weights are created when the Model is first called on inputs or `build()` is called with an `input_shape`.


In [18]:
try:
    model.summary()
except Exception as e:
    print(e)

This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.


In [19]:
x = tf.ones((1, 4))
y = model(x)
model.weights  # y = x * weights

[<tf.Variable 'dense_4/kernel:0' shape=(4, 2) dtype=float32, numpy=
 array([[-0.17153788,  0.2804985 ],
        [ 0.22205567,  0.97227025],
        [ 0.5218835 ,  0.13220286],
        [-0.24572921, -0.48590803]], dtype=float32)>,
 <tf.Variable 'dense_4/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>,
 <tf.Variable 'dense_5/kernel:0' shape=(2, 3) dtype=float32, numpy=
 array([[-0.5866658 ,  0.06398404,  0.2683009 ],
        [ 0.9433899 ,  0.3961513 ,  0.49218595]], dtype=float32)>,
 <tf.Variable 'dense_5/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>,
 <tf.Variable 'dense_6/kernel:0' shape=(3, 4) dtype=float32, numpy=
 array([[-0.5344628 , -0.0570069 , -0.7125208 ,  0.8921783 ],
        [ 0.2873832 ,  0.38166165, -0.8393112 , -0.9051145 ],
        [ 0.87742424,  0.70356834, -0.40055728, -0.83361983]],
       dtype=float32)>,
 <tf.Variable 'dense_6/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]

In [20]:
len(model.weights)

6

In [21]:
model.summary() # 10 = 4 * 2 + 2; 9 = 2 * 3 + 3; 16 = 3 * 4 + 4

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (1, 2)                    10        
_________________________________________________________________
dense_5 (Dense)              (1, 3)                    9         
_________________________________________________________________
dense_6 (Dense)              (1, 4)                    16        
Total params: 35
Trainable params: 35
Non-trainable params: 0
_________________________________________________________________


In [22]:
# 以Input开始创建Sequential model,
model = keras.Sequential()
model.add(keras.Input(shape=(4,)))
model.add(layers.Dense(2, activation="relu"))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 2)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Input不作为layer出现在模型结构里
model.layers

In [24]:
# 也可以在第一层传入input_shape参数
model = tf.keras.Sequential()
model.add(layers.Dense(2, activation='relu', input_shape=(4,)))

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 2)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


## 常用的调试工作流：add() + summary()
- 创建Sequential model后可通过add()添加层和使用summary()查看模型结构进行调试

In [25]:
model = keras.Sequential()
model.add(keras.Input(shape=(250, 250, 3)))  # 250x250 RGB images
model.add(layers.Conv2D(32, 5, strides=2, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))

# Can you guess what the current output shape is at this point? Probably not.
# Let's just print it:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 123, 123, 32)      2432      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 121, 121, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 40, 40, 32)        0         
Total params: 11,680
Trainable params: 11,680
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(2))

# And now?
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 123, 123, 32)      2432      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 121, 121, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 40, 40, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 38, 38, 32)        9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 36, 36, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 10, 10, 32)       

In [27]:
# Now that we have 4x4 feature maps, time to apply global max pooling.
model.add(layers.GlobalMaxPooling2D())

# Finally, we add a classification layer.
model.add(layers.Dense(10))

In [29]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 123, 123, 32)      2432      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 121, 121, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 40, 40, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 38, 38, 32)        9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 36, 36, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 10, 10, 32)       

## 提取序贯模型的特征属性
- 序贯模型像函数式API模型一样，可直接调用起输入输出或者其他属性

In [30]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)

In [31]:
feature_extractor = keras.Model(inputs=initial_model.inputs, 
                                outputs=[layer.output for layer in initial_model.layers])
feature_extractor.summaryary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 250, 250, 3)]     0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 123, 123, 32)      2432      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 121, 121, 32)      9248      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 119, 119, 32)      9248      
Total params: 20,928
Trainable params: 20,928
Non-trainable params: 0
_________________________________________________________________


In [32]:
initial_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 123, 123, 32)      2432      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 121, 121, 32)      9248      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 119, 119, 32)      9248      
Total params: 20,928
Trainable params: 20,928
Non-trainable params: 0
_________________________________________________________________


In [33]:
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)

In [36]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu", name="my_intermediate_layer"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)

feature_extr2 = keras.Model(inputs=initial_model.inputs, 
                            outputs=initial_model.get_layer(name="my_intermediate_layer").output)
feature_extr2.summary()
x = tf.ones((1, 250, 250, 3))
features = feature_extr2(x)

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 250, 250, 3)]     0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 123, 123, 32)      2432      
_________________________________________________________________
my_intermediate_layer (Conv2 (None, 121, 121, 32)      9248      
Total params: 11,680
Trainable params: 11,680
Non-trainable params: 0
_________________________________________________________________


In [37]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu", name="my_intermediate_layer"),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)

feature_extr3 = keras.Model(inputs=initial_model.inputs, 
                            outputs=initial_model.get_layer(name="my_intermediate_layer").output)
feature_extr3.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 250, 250, 3)]     0         
_________________________________________________________________
my_intermediate_layer (Conv2 (None, 123, 123, 32)      2432      
Total params: 2,432
Trainable params: 2,432
Non-trainable params: 0
_________________________________________________________________


In [38]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Conv2D(32, 3, activation="relu", name="my_intermediate_layer"),
    ]
)

feature_extr4 = keras.Model(inputs=initial_model.inputs, 
                            outputs=initial_model.get_layer(name="my_intermediate_layer").output)
feature_extr4.summary()

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 250, 250, 3)]     0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 123, 123, 32)      2432      
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 121, 121, 32)      9248      
_________________________________________________________________
my_intermediate_layer (Conv2 (None, 119, 119, 32)      9248      
Total params: 20,928
Trainable params: 20,928
Non-trainable params: 0
_________________________________________________________________


In [39]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu", name='layer1'),
        layers.Conv2D(32, 3, activation="relu", name='layer2'),
        layers.Conv2D(32, 3, activation="relu", name="layer3"),
    ]
)

feature_extr4 = keras.Model(inputs=initial_model.inputs, 
                            outputs=initial_model.get_layer(name="layer2").output)
feature_extr4.summary()

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 250, 250, 3)]     0         
_________________________________________________________________
layer1 (Conv2D)              (None, 123, 123, 32)      2432      
_________________________________________________________________
layer2 (Conv2D)              (None, 121, 121, 32)      9248      
Total params: 11,680
Trainable params: 11,680
Non-trainable params: 0
_________________________________________________________________


## 序贯模型进行迁移学习

In [41]:
model = keras.Sequential([
    keras.Input(shape=(784)),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(10),
])
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 32)                25120     
_________________________________________________________________
dense_11 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_12 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_13 (Dense)             (None, 10)                330       
Total params: 27,562
Trainable params: 27,562
Non-trainable params: 0
_________________________________________________________________


In [42]:
# 通过layer.trainable=False设置, 冻结除了最后一层的所有层的权重， 然后重新compile和train更新最后一层的权重
for layer in model.layers[:-1]:
    layer.trainable = False

In [43]:
model.summary()  # 注意可训练参数

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 32)                25120     
_________________________________________________________________
dense_11 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_12 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_13 (Dense)             (None, 10)                330       
Total params: 27,562
Trainable params: 330
Non-trainable params: 27,232
_________________________________________________________________


In [44]:
# 通过预训练模型和新加的分类层堆叠成序贯模型
# Load a convolutional base with pre-trained weights
base_model = keras.applications.Xception(
    weights='imagenet',
    include_top=False,
    pooling='avg')

# Freeze the base model
base_model.trainable = False

83689472/83683744 [==============================] - 56s 1us/step


In [45]:
# Use a Sequential model to add a trainable classifier on top
model = keras.Sequential([
    base_model,
    layers.Dense(1000),
])

In [46]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 2048)              20861480  
_________________________________________________________________
dense_14 (Dense)             (None, 1000)              2049000   
Total params: 22,910,480
Trainable params: 2,049,000
Non-trainable params: 20,861,480
_________________________________________________________________
